In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df3 = pd.read_excel('/content/drive/MyDrive/Project Data/X-ray1.xlsx')
df3.drop(df3.columns[df3.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [ ]:
df3.head(2)

,Patient_ID,Image_file1,Image_file2,Report_ID,Comparison,Indication,Findings,Impression
0,CXR1,CXR1_1_IM-0001-4001.png,CXR1_1_IM-0001-3001.png,1.xml,None.,Positive TB test,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,CXR2,CXR2_IM-0652-1001.png,CXR2_IM-0652-2001.png,2.xml,None.,Preop bariatric surgery.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.


In [ ]:
df3['Impression'].value_counts()

No acute cardiopulmonary abnormality.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                301
No active disease.                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [ ]:
df3.drop(['Comparison', 'Indication','Findings'], axis=1, inplace=True)

In [ ]:
df3

,Patient_ID,Image_file1,Image_file2,Report_ID,Impression
0,CXR1,CXR1_1_IM-0001-4001.png,CXR1_1_IM-0001-3001.png,1.xml,Normal chest x-XXXX.
1,CXR2,CXR2_IM-0652-1001.png,CXR2_IM-0652-2001.png,2.xml,No acute pulmonary findings.
2,CXR3,CXR3_IM-1384-1001.png,CXR3_IM-1384-2001.png,3.xml,"No displaced rib fractures, pneumothorax, or p..."
3,CXR4,CXR4_IM-2050-2001.png,CXR4_IM-2050-1001.png,4.xml,1. Bullous emphysema and interstitial fibrosis...
4,CXR5,CXR5_IM-2117-1004003.png,CXR5_IM-2117-1003002.png,5.xml,No acute cardiopulmonary abnormality.
...,...,...,...,...,...
3815,CXR3995,CXR3995_IM-2046-2001.png,CXR3995_IM-2046-1001.png,3995.xml,1. Interval resolution of bibasilar airspace d...
3816,CXR3996,CXR3996_IM-2047-3001.png,CXR3996_IM-2047-1001.png,3996.xml,Clear lungs. No acute cardiopulmonary abnormal...
3817,CXR3997,CXR3997_IM-2048-1001.png,CXR3997_IM-2048-1002.png,3997.xml,"No acute findings, no evidence for active TB."
3818,CXR3998,CXR3998_IM-2048-1002.png,CXR3998_IM-2048-1001.png,3998.xml,Heart size is normal and the lungs are clear.


In [ ]:
df3['Impression']= df3['Impression'].str.lower()

In [ ]:
df3['Impression'].str.contains('x-xxxx').sum()

67

In [ ]:
df3[df3['Impression'].str.contains('x-xxxx')].index

Int64Index([   0,   44,  107,  119,  208,  210,  223,  388,  489,  528,  673,
             844,  971, 1101, 1104, 1115, 1150, 1210, 1240, 1358, 1400, 1539,
            1627, 1701, 1843, 1845, 1877, 1900, 2004, 2048, 2049, 2137, 2145,
            2194, 2217, 2218, 2233, 2272, 2326, 2356, 2436, 2517, 2562, 2660,
            2709, 2784, 2826, 2850, 2906, 3019, 3138, 3183, 3222, 3227, 3236,
            3284, 3334, 3345, 3346, 3395, 3487, 3504, 3539, 3608, 3634, 3694,
            3701],
           dtype='int64')

In [ ]:
df3['Impression'] = df3['Impression'].str.replace('x-xxxx', '')

In [ ]:
df3.shape

(3820, 5)

In [ ]:
df3['Impression'].str.contains('x-xxxx').sum()

0

In [ ]:
df3['Impression'].str.contains('xxxx').sum()

595

In [ ]:
df3['Impression'] = df3['Impression'].str.replace('xxxx', '')

In [ ]:
df3['Impression'].str.contains('xxxx').sum()

0

In [ ]:
punctuations = list(string.punctuation)
for i in punctuations:
        df3['Impression'] = df3['Impression'].str.replace(i, '')
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df3['Impression'].str.contains('1').sum()

800

In [ ]:
for j in range(0,len(df3)):
    imp = df3['Impression'][j]    
    rem_nos = str.maketrans('', '', digits)
    df3['Impression'][j]  = imp.translate(rem_nos)

In [ ]:
df3['Impression'].str.contains('1').sum()

0

In [ ]:
# Extra spaces#
for j in range(0,len(df3)):
    exsp = df3['Impression'][j]
    exsp = ' '.join(exsp.split())
    df3['Impression'][j] = exsp

In [ ]:
df3['Impression_token'] = 'BEGIN' + df3.Impression + 'END'

In [ ]:
df3['Impression'].value_counts()

no acute cardiopulmonary abnormality                                                                                                                                                                                                                                                                                                                                                                                                                                                                     411
no acute cardiopulmonary findings                                                                                                                                                                                                                                                                                                                                                                                                                                                                        181
no acute cardi

In [ ]:
l= df3['Impression'].value_counts().tolist()

In [ ]:
l

[411,
 181,
 160,
 136,
 132,
 110,
 92,
 89,
 83,
 63,
 58,
 57,
 31,
 30,
 29,
 29,
 28,
 28,
 27,
 21,
 20,
 19,
 18,
 17,
 16,
 16,
 15,
 15,
 11,
 10,
 9,
 9,
 9,
 8,
 8,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
from PIL import Image
  
# get image
img = Image.open(r'/content/drive/MyDrive/Project Data/NLMCXR_png/CXR1_1_IM-0001-3001.png')
  
# get width and height
width = img.width
height = img.height
  
# display width and height
print("The height of the image is: ", height)
print("The width of the image is: ", width)

The height of the image is:  624
The width of the image is:  512


In [ ]:
df3.to_excel('/content/drive/MyDrive/Project Data/X-ray2.xlsx')